# 桩号预测

In [ ]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

from config import *
context = Context('validation', gpu_start=4, dep_var=['桩号'], debug=0, mulr=1, fn_feather='tbmData/feather/removed_redundent1-full-30-no_noise-zhuanghao.feather')
databunch = MultiDeptTabularDataBunch.from_df('tmp', context.cyc_cont, context.dep_var, valid_idx=context.valid_idx_tile, bs=context.bs)

preload_pytorch()
record = False

In [ ]:
hyper_params = {
    'n_hidden': 100,
    'n_layers': 3,
    'hidden_p': 0.3,
    'input_p': 0.6,
    'weight_p': 0.5,
    'layers': [3*context.n_cont, 5, len(context.dep_var)],
    'drops': [0.2] * 3,
    'alpha': 2.,
    'beta': 1.,
    'clip': 0.5,
#     'loss_func' = MSELossFlat()
    'loss_func': l1
}

learner = get_cont_model(context, databunch, hyper_params)

In [ ]:
learner.load('removed_redundent1-full-30-7-zhuanghao');

In [ ]:
xb,yb = learner.data.one_batch(DatasetType.Valid, detach=False, denorm=False)
#     xb,yb = learner.data.one_batch(DatasetType.Train, detach=False, denorm=False)
# remove noise
# xb = [o[::context.mulr] for o in xb]
# yb = yb[::context.mulr]

yb = to_detach(yb)
cb_handler = CallbackHandler(learner.callbacks) # rnn trainer
learner.model.reset()
pb, _ = loss_batch(learner.model.eval(), xb, yb, cb_handler=cb_handler)
loss, mapd = learner.loss_func(pb, yb), learner.metrics[0](pb, yb)
if 1:
    print('Predicting mean: mapd =', learner.metrics[0](pb, pb.mean(0)))
print('mapd', to_np(mapd))
y_np, p_np = (to_np(o).squeeze() for o in (yb,pb))

# Plot p against y. Expect linearity
yb_denormed, pb_denormed = denormalize(y_np, *context.stat_y), denormalize(p_np, *context.stat_y)
df = pd.DataFrame({'y': yb_denormed, 'p': pb_denormed})
sns.jointplot(x='y', y='p', data=df, kind="reg")

# reshape x
# xb_r = to_np(xb[1]).reshape(-1,len(context.cont_names),context.sl)

In [ ]:
df = pd.DataFrame({'y': yb_denormed, 'p': pb_denormed})

In [ ]:
context.stat_y

In [ ]:
from crits import MAPD

In [ ]:
mapd = MAPD(stats=context.stat_y)

In [ ]:
mapd(yb, yb.mean(0))

In [ ]:
mapd(yb, pb)

In [ ]:
# load original data
fmtr = DataFormatter(context, cycle_feathers=context.fn_cycles)
cycles = fmtr.cycles # shorten notation

In [ ]:
xb_or = context.cyc_cont.values[:,:-2].reshape(context.cyc_cont.shape[0],len(context.cont_names),context.sl)

In [ ]:
learner.data.valid_ds[i][0].data[1].shape

In [ ]:
xb = np.array([learner.data.valid_ds[i][0].data[1].numpy() for i in range(5)])
# reshape x
xb_r = xb.reshape(-1,len(context.cont_names),context.sl)

In [ ]:
i = 500
(xb_r[:i] - xb_or[:i]).sum()

Problem here. The validation samples is not the first few samples of the dataset

In [ ]:
def validate(learner):
    xb,yb = learner.data.one_batch(DatasetType.Valid, detach=False, denorm=False)
#     xb,yb = learner.data.one_batch(DatasetType.Train, detach=False, denorm=False)
    
    yb = to_detach(yb)
    cb_handler = CallbackHandler(learner.callbacks) # rnn trainer
    learner.model.reset()
    pb, _ = loss_batch(learner.model.eval(), xb, yb, cb_handler=cb_handler)
    loss, mapd = learner.loss_func(pb, yb), learner.metrics[0](pb, yb)
    y_np, p_np = (to_np(o) for o in (yb,pb))
    
    # Plot p against y. Expect linearity
    yb_denormed, pb_denormed = denormalize(y_np, *context.stat_y), denormalize(p_np, *context.stat_y)
    i=0;sns.jointplot(x=yb_denormed[:,i], y=pb_denormed[:,i], kind="reg")
    i=1;sns.jointplot(x=yb_denormed[:,i], y=pb_denormed[:,i], kind="reg")
    
    return loss, mapd

In [ ]:
loss, mapd = validate(learner); mapd

Compare X of different noise

In [ ]:
def reconstruct(x, context):
    if isinstance(x, Tensor): x = to_np(x)
    x = x.reshape(context.sl, len(context.cont_names))
    return pd.DataFrame(x, columns=context.cont_names)

In [ ]:
# without noise
ax = None
for i in range(3):
    df = reconstruct(xb_np[i], context)
    ax = plots(df, ax=ax)